In [1]:
import pandas as pd

In [2]:
Restaurent_Data = pd.read_csv("C:\\Users\\graykar\\Downloads\\Pyhton\\Data Files\\ResturentSentiment_Analysis.tsv", sep='\t')

In [3]:
pd.set_option('max_colwidth', 200)

In [4]:
Restaurent_Data.head(10)

,Review,Liked
0,I love it.,1
1,I don't Love it...,0
2,I do not Love it...,0
3,Wow... Loved this place.,1
4,Crust is not good.,0
5,Not tasty and the texture was just nasty.,0
6,Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
7,The selection on the menu was great and so were the prices.,1
8,Now I am getting angry and I want my damn pho.,0
9,Honeslty it didn't taste THAT fresh.),0


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\graykar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
Restaurent_Data.shape[0]

1003

In [25]:
converted_data = []
LMT = WordNetLemmatizer()

for i in range(Restaurent_Data.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', Restaurent_Data['Review'][i] ).lower().split() 
    review = [LMT.lemmatize(x) for x in review if not x in set(stopwords.words('english')) - {'not'}]
    review = " ".join(review)
    converted_data.append(review)

In [10]:
from tensorflow.keras.preprocessing.text import  one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import numpy as np

C:\Users\graykar\Documents\Annaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [11]:
vocabulary_size = 10000

In [26]:
one_hot_representation = [one_hot(word,vocabulary_size) for word in converted_data]

In [27]:
for i in one_hot_representation:
    if len(i) > 15:
        print(len(i))
    else:
        continue

18
17
17
20


In [14]:
string_lenght = 20

In [28]:
Embeded_doc = pad_sequences(one_hot_representation, maxlen=string_lenght, padding='pre')
print(Embeded_doc)

[[   0    0    0 ...    0    0 7582]
 [   0    0    0 ...    0    0 7582]
 [   0    0    0 ...    0 8320 7582]
 ...
 [   0    0    0 ... 8320 7119 3904]
 [   0    0    0 ... 3450 4203 5028]
 [   0    0    0 ... 4160 6662 2051]]


In [35]:
from tensorflow.keras.layers import Dropout

In [36]:
vec_dimension = 25
model = Sequential()
model.add(Embedding(vocabulary_size, vec_dimension , input_length=string_lenght ))
model.add(Dropout(0.25))
model.add(LSTM(100))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 25)            250000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 25)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               50400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 300,501
Trainable params: 300,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
print(len(Embeded_doc)) , Restaurent_Data['Liked'].shape

1003


(None, (1003,))

In [30]:
X_final = np.array(Embeded_doc)
Y_final = np.array(Restaurent_Data['Liked'])

In [31]:
X_final.shape , Y_final.shape

((1003, 20), (1003,))

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X_final , Y_final , test_size=0.25, random_state=48 )

In [33]:
X_train.shape, y_train.shape,  X_test.shape, y_test.shape

((752, 20), (752,), (251, 20), (251,))

In [37]:
model.fit(X_train , y_train , epochs=25 , validation_data=(X_test,y_test), batch_size=8 )

Train on 752 samples, validate on 251 samples
Epoch 1/25
752/752 [==============================] - 4s 5ms/sample - loss: 0.6898 - accuracy: 0.5266 - val_loss: 0.6784 - val_accuracy: 0.6335
Epoch 2/25
752/752 [==============================] - 1s 2ms/sample - loss: 0.5491 - accuracy: 0.7739 - val_loss: 0.5102 - val_accuracy: 0.7928
Epoch 3/25
752/752 [==============================] - 1s 2ms/sample - loss: 0.3060 - accuracy: 0.8976 - val_loss: 0.5202 - val_accuracy: 0.7371
Epoch 4/25
752/752 [==============================] - 2s 2ms/sample - loss: 0.1716 - accuracy: 0.9441 - val_loss: 0.5189 - val_accuracy: 0.7331
Epoch 5/25
752/752 [==============================] - 1s 2ms/sample - loss: 0.1057 - accuracy: 0.9668 - val_loss: 0.5977 - val_accuracy: 0.7809
Epoch 6/25
752/752 [==============================] - 1s 2ms/sample - loss: 0.0826 - accuracy: 0.9774 - val_loss: 0.5937 - val_accuracy: 0.7610
Epoch 7/25
752/752 [==============================] - 1s 2ms/sample - loss: 0.0536 - accur

In [40]:
y_pred = model.predict_classes(X_test)

In [41]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[112  26]
 [ 27  86]]


In [43]:
from sklearn.metrics import f1_score, accuracy_score
f1_score(y_test, y_pred)
accuracy_score (y_test, y_pred)

0.7888446215139442